In [1]:
!pip install -U statsmodels

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [3]:
df = pd.read_csv('C:/Users/Kazem Kamrani/Desktop/Time Series Analysis/international-airline-passengers.csv',
                 index_col = 'Month',
                 parse_dates = True)

In [4]:
df.index.freq = 'MS'

In [5]:
df.shape

(144, 1)

In [6]:
# Assume the forecast horizon we care about is 12 
# Validate over 10 steps
h = 12 
steps = 10 
Ntest = len(df) - h - steps + 1 

In [7]:
# Configuration hyperparameters to try 
trend_type_list = ['add','mul']
seasonal_type_list = ['add','mul']
damped_trend_list = [True, False]
init_method_list = ['estimated','heuristic','legacy-heuristic']
use_boxcox_list = [True,False, 0]

In [8]:
# NOTE: Statsmodels documentation stats that 'log' is an acceptable input
#       for use_boxcox. this is false.

In [9]:
def walkforward(
    trend_type,
    seasonal_type,
    damped_trend,
    init_method,
    use_boxcox,
    debug=False):
    
  # store errors
  errors = []
  seen_last = False
  steps_completed = 0 

  for end_of_train in range(Ntest, len(df) - h + 1 ): 
    
      train = df.iloc[:end_of_train]
      test = df.iloc[end_of_train:end_of_train + h]
    
      if test.index[-1]:
          seen_last = True
        
      steps_completed +=1
    
      hw = ExponentialSmoothing(
          train['Passengers'],
          initialization_method=init_method,
          trend=trend_type,
          damped_trend=damped_trend,
          seasonal =seasonal_type,
          seasonal_periods=12,
          use_boxcox=use_boxcox)
      res_hw = hw.fit()
      # compute error for the forecast horizon
      fcast = res_hw.forecast(h)
      error = mean_squared_error(test['Passengers'], fcast)
      errors.append(error)
  if debug:
    print("seen_last:", seen_last)
    print("steps completed:", steps_completed)
    
  return np.mean(errors)

In [10]:
# test our function 
walkforward('add','add',False,'legacy-heuristic', 0, debug=True)

seen_last: True
steps completed: 10


2521.0305142383577

In [11]:
# Interate through all possible options (i.e. grid search)
tuple_of_option_lists = (
    trend_type_list,
    seasonal_type_list,
    damped_trend_list,
    init_method_list,
    use_boxcox_list,
)
for x in itertools.product(*tuple_of_option_lists):
    print(x)
    

('add', 'add', True, 'estimated', True)
('add', 'add', True, 'estimated', False)
('add', 'add', True, 'estimated', 0)
('add', 'add', True, 'heuristic', True)
('add', 'add', True, 'heuristic', False)
('add', 'add', True, 'heuristic', 0)
('add', 'add', True, 'legacy-heuristic', True)
('add', 'add', True, 'legacy-heuristic', False)
('add', 'add', True, 'legacy-heuristic', 0)
('add', 'add', False, 'estimated', True)
('add', 'add', False, 'estimated', False)
('add', 'add', False, 'estimated', 0)
('add', 'add', False, 'heuristic', True)
('add', 'add', False, 'heuristic', False)
('add', 'add', False, 'heuristic', 0)
('add', 'add', False, 'legacy-heuristic', True)
('add', 'add', False, 'legacy-heuristic', False)
('add', 'add', False, 'legacy-heuristic', 0)
('add', 'mul', True, 'estimated', True)
('add', 'mul', True, 'estimated', False)
('add', 'mul', True, 'estimated', 0)
('add', 'mul', True, 'heuristic', True)
('add', 'mul', True, 'heuristic', False)
('add', 'mul', True, 'heuristic', 0)
('add

In [12]:
best_score = float('inf')
best_options = None
for x in itertools.product(*tuple_of_option_lists):
    score = walkforward(*x)
    
    if score < best_score:
        print("best score so far:", score)
        best_score = score 
        best_options = x 

best score so far: 412.81720294821025
best score so far: 412.7068213361361
best score so far: 320.6640771493606


C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\sit

best score so far: 303.71164998411547


C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\sit

C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\sit

best score so far: 263.33618316574746


C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
C:\Users\Kazem Kamrani\anaconda3\lib\sit

In [13]:
print("best score:", best_score)

trend_type, seasonal_type, damped_trend, init_method, use_boxcox = best_options 
print("trend_type", trend_type)
print("seasonal_type",seasonal_type)
print("damped_trend", damped_trend)
print("init_mrthod", init_method)
print("use_boxcox", use_boxcox)

best score: 263.33618316574746
trend_type mul
seasonal_type mul
damped_trend True
init_mrthod legacy-heuristic
use_boxcox False
